In [2]:
import numpy as np
import pickle
import os
import download
from dataset import one_hot_encoded
import cifar10

In [3]:
%cd '/content/drive/MyDrive/Colab Notebooks'

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks'
/content


In [4]:
cifar10.data_path = "data/CIFAR-10/Desktop"

In [5]:
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [6]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [7]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/test_batch


In [9]:
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score,KFold,GridSearchCV

In [10]:
x_train=images_train.reshape(50000,3072)
y_train=cls_train
x_test=images_test.reshape(10000,3072)
y_test=cls_test

In [11]:
cifar_pca=PCA()
cifar_pca.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [12]:
k=0
total=0
while total<0.95:
    total+=cifar_pca.explained_variance_ratio_[k]
    k=k+1
k    

217

In [13]:
pca_cifar=PCA(n_components=k,whiten=True)
x_pca_train=pca_cifar.fit_transform(x_train)
x_pca_test=pca_cifar.transform(x_test)

In [20]:
clf = svm.SVC()
clf.fit(x_pca_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [22]:
y_pred=clf.predict(x_pca_test)

In [23]:
class_name=[]
for i in y_pred:
    class_name.append(class_names[i])

In [24]:
class_name=np.array(class_name)

In [25]:
np.savetxt('pred.csv',class_name, fmt='%s', delimiter=',', encoding=None)

In [21]:
clf.score(x_pca_test,y_test)

0.5533